# Demonstration of the lowEBMs Package

-----

- Installation
- Import
- Configuration
- Simulation

For more information, see https://lowebms.readthedocs.io/en/latest/

## Installation
----
In your linux terminal: __pip install lowEBMs__ 

Alternatively use the github-repository: https://github.com/BenniSchmiedel/Low-dimensional-EBMs

## Import
----
There are some modules which have to be imported. The core modules of any simulation run with the __lowEBMs__ are:

In [ ]:
from lowEBMs.Packages.Configuration import importer, add_sellersparameters, parameterinterpolatorstepwise 
from lowEBMs.Packages.Variables import variable_importer, Vars, trackerreset
from lowEBMs.Packages.RK4 import rk4alg
from lowEBMs.Packages.ModelEquation import model_equation

To allow calculations use __numpy__ and for plotting for example __matplotlib__ :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

There are some other useful additional features in **lowEBMs**, for example:

- the ``Constants`` module with physical constants and conversions
- the ``update_plotstyle`` function to create nicer plots
- the ``moving_average`` function to smooth data with a moving average if needed

In [ ]:
import lowEBMs.Packages.Constants as const
from lowEBMs import update_plotstyle, moving_average
update_plotstyle()
import matplotlib
matplotlib.rcParams['lines.linewidth']=2

## Configuration
---

The model is configured through an external __configuration.ini-file__! It includes:

- The specifications of the numerical integration
- The structure of the model equation
- The initial condition
- The parameters of the physical equation

# Demonstration 0D EBM
----


### Import the configuration with 

``importer()``

### Import variables and initial conditions with

``variable_importer()``


### Run the simulation with 

``rk4alg()``

In [ ]:
configuration_OD_Spinup=importer('Demonstration_config_0D_Spinup.ini')

variable_importer(configuration_OD_Spinup,control=True,accuracy_number=10000)

Time_Spinup,ZMT_Spinup,GMT_Spinup=rk4alg(model_equation,configuration_OD_Spinup['eqparam'],configuration_OD_Spinup['rk4input'],configuration_OD_Spinup['funccomp'])

In [ ]:
plt.plot(   Time_Spinup/const.time_sec_year,   GMT_Spinup)

In [ ]:
configuration_Crowley_OD=importer('Demonstration_config_Crowley_0D.ini')

variable_importer(configuration_Crowley_OD)

Vars.T,Vars.T_global=ZMT_Spinup[-1],GMT_Spinup[-1]
Time_Crowley_0D,ZMT_Crowley_0D,GMT_Crowley_0D=rk4alg(model_equation,configuration_Crowley_OD['eqparam'],configuration_Crowley_OD['rk4input'],configuration_Crowley_OD['funccomp'])

In [ ]:
import lowEBMs.Packages.Constants as const
from lowEBMs import update_plotstyle, moving_average
update_plotstyle()
import matplotlib
matplotlib.rcParams['lines.linewidth']=2

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(16,7))

plt.plot( Time_Crowley_0D/const.time_sec_year , GMT_Crowley_0D-GMT_Crowley_0D[0] ,\
         label='Late Holocene 0DEBM (CRW+SBF)' , c='#e15759' )


Time_HadCM3,GMT_HadCM3=np.loadtxt('Data/HadCM3_GMT_smoothed.txt',delimiter=',',unpack=True)
plt.plot(Time_HadCM3,GMT_HadCM3, label='Late Holocene HadCM3 (CRW+SBF)',c='#76b7b2', linewidth=1.5)


ax.set_xlim(850,1850),ax.set_xticks(np.linspace(850,1850,6))
ax.set_ylim(-1.5,1),ax.set_xlabel('$Year$'),ax.set_ylabel('$GMT_{response}\; [K]$')
plt.legend(fontsize=16,loc='lower right')

In [ ]:
configuration_EVA_OD=importer('Demonstration_config_EVA_0D.ini')

variable_importer(configuration_EVA_OD)

Vars.T,Vars.T_global=ZMT_Spinup[-1],GMT_Spinup[-1]
Time_EVA_0D,ZMT_EVA_0D,GMT_EVA_0D=rk4alg(model_equation,configuration_EVA_OD['eqparam'],configuration_EVA_OD['rk4input'],configuration_EVA_OD['funccomp'])

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(16,7))

plt.plot( Time_Crowley_0D/const.time_sec_year , GMT_Crowley_0D-GMT_Crowley_0D[0] ,\
         label='Late Holocene 0DEBM (CRW+SBF)' , c='#e15759' )

plt.plot( Time_EVA_0D/const.time_sec_year , GMT_EVA_0D-GMT_Crowley_0D[0] ,\
         label='Late Holocene 0DEBM (Sigl+SBF)' ,c='#76b7b2' )

ax.set_xlim(850,1850),ax.set_xticks(np.linspace(850,1850,6))
ax.set_ylim(-1.5,0.1),ax.set_xlabel('$Year$'),ax.set_ylabel('$GMT_{response}\; [K]$')
plt.legend(fontsize=16,loc='lower right')

# Demonstration 1D EBM
---

This simulation runs with the Sellers energy transfer flux which requires parameters resolved over the latitudes. 

Those are stored externally and can be imported with ``add_sellersparameters()``.

In [ ]:
configuration_1D=importer('Demonstration_config_1D.ini')

variable_importer(configuration_1D,control=True)

configuration_1D,ParasCTRL=add_sellersparameters(configuration_1D,\
                                    parameterinterpolatorstepwise,'SellersParameterization.ini',2,0,True,True)

Time_1D,ZMT_1D,GMT_1D=rk4alg(model_equation,configuration_1D['eqparam'],configuration_1D['rk4input'],configuration_1D['funccomp'])

In [ ]:
from lowEBMs.Packages.Variables import Vars

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(16,7))


ax[0].plot(Time_1D[1:]/const.time_sec_year,GMT_1D[1:],c='#4e79a7')
ax[0].scatter(Time_1D[-1]/const.time_sec_year,GMT_1D[-1],c='#4e79a7')

ax[0].set_xlabel('$Year$'),ax[0].set_ylabel('$GMT\; [K]$')
ax[0].set_title('GMT 1D-EBM Spinup')

ax[1].plot(Vars.Lat,ZMT_1D[-1],c='#f28e2b')

ax[1].set_xlabel('$Latitudes\; [°]$'),ax[1].set_ylabel('$ZMT\; [K]$')
ax[1].set_xticks(np.linspace(-90,90,7))
ax[1].set_title('ZMT 1D-EBM Spinup')